# Подготовка датасетов

Мы собрали несколько датасетов на русском языке, направленных преимущественно на задачи STS, среди которых:
- https://huggingface.co/datasets/sberquad
- https://huggingface.co/datasets/RussianNLP/russian_super_glue
<!-- - https://github.com/vladislavneon/RuBQ/tree/master/RuBQ_2.0 -->



# Подготовка датасета для обучения

In [ ]:
import random
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset_RSG_danetqa = load_dataset("RussianNLP/russian_super_glue", name='danetqa')
dataset_RSG_muserc = load_dataset("RussianNLP/russian_super_glue", name='muserc')
dataset_SQ = load_dataset("sberquad")

In [ ]:
train_data = pd.DataFrame()

In [ ]:
muserc = pd.concat([dataset_RSG_muserc['train'].to_pandas().rename(columns = {'paragraph':'context'}), 
                    dataset_RSG_muserc['validation'].to_pandas().rename(columns = {'paragraph':'context'})])

In [ ]:
muserc = muserc[['question', 'context']]

In [ ]:
muserc.size

In [ ]:
muserc.drop_duplicates(inplace=True)

In [ ]:
muserc['label'] = 1

In [ ]:
muserc['context'] = muserc['context'].str.replace(r'\(\d+\)\s*', '', regex=True)

In [ ]:
danetqa = pd.concat([dataset_RSG_danetqa['train'].to_pandas()[['question', 'passage', 'label']].rename(columns = {'passage':'context'}), 
                     dataset_RSG_danetqa['validation'].to_pandas()[['question', 'passage', 'label']].rename(columns = {'passage':'context'})])

In [ ]:
print(danetqa.shape)
print(muserc.shape)

In [ ]:
train_data = pd.concat([train_data,
                        muserc,
                        danetqa], ignore_index=True)

In [ ]:
train_data.shape

In [ ]:
# берем данные из датасетов, в которых все пары вопрос-ответ правильные
to_shafle = pd.concat([dataset_SQ,
                       muserc], ignore_index=True)
to_shafle.shape

### Negative sampling

Удаляем повторяющиеся контексты, чтобы при перемешивании не было правильных вариантов

Перемешиваем ответы на вопрос так, чтобы в соответствие каждому вопросу был поставлен неправильный(случайный) ответ

In [ ]:
shuffled_context = to_shafle['context'].to_list()
random.shuffle(shuffled_context)
to_shafle['shuffled_context'] = shuffled_context

In [ ]:
to_shafle[to_shafle['context']==to_shafle['shuffled_context']]

In [ ]:
# Удалить повторяющиеся колонки при необходимости

to_shafle = to_shafle.drop([24165, 35062])

Всем строкам ставим label=0

In [ ]:
to_shafle['label'] = 0
to_shafle.drop(columns=['context'], inplace=True)
to_shafle.rename(columns={'shuffled_context': 'context'}, inplace=True)
to_shafle.head()

In [ ]:
to_shafle.shape

Объединяем датасет с правильными и неправильными парами вопрос-ответ

In [ ]:
train_data = pd.concat([train_data,
                        to_shafle], ignore_index=True)
train_data.shape

In [ ]:
train_data['label'].value_counts()

In [ ]:
# Сохраним датасет

train_data.to_csv('train_data.csv', index=False)